# VNAxKEPCO Data Analysis Tutorial

Welcome to the VNAxKEPCO Data Analysis Tutorial! This guide will walk you through the process of analyzing data from your VNAxKEPCO experiments using the provided Python script. The tutorial is divided into sections to cover different aspects of the analysis.

## Prerequisites

Before you start, make sure you have the required software and libraries installed:

- Python: Install Python from [python.org](https://www.python.org/downloads/).
- Jupyter Notebook: Install Jupyter Notebook by running `pip install jupyter` in your terminal.
- Required Libraries: Install the necessary Python libraries using the following command:

```bash
pip install numpy matplotlib pandas scipy pyvisa
```

## Section 1: 1 Port Experiment

### 1.1 Initializing 1 Port Experiment

```python
from VNAxKEPCO_1P import Experiment1P

E1 = Experiment1P()
```

### 1.2 Frequency Sweep

Customize the experiment parameters like frequency range, fields, and sample name.

```python
ref_field = 400
start_freq = 0.5
stop_freq = 6
step = 20e6  # 20 MHz
start_freq, stop_freq = start_freq * 1e9, stop_freq * 1e9
num_points = int((stop_freq - start_freq) / step) + 1
fields = np.linspace(0, 400, 201)

# Change this
sample = 'NiFe'
file_name = f'1_Port_{sample}_{start_freq/1e9}-{stop_freq/1e9}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\physlab\Desktop\VNA Automation\Data\{file_name}'

# Run the experiment
E1.MeasureRef(ref_field, start_freq, stop_freq, num_points, dir)
E1.Measure(start_freq, stop_freq, num_points, fields, dir)
```

### 1.3 Data Loading

```python
dir = r'c:\Users\physlab\Desktop\VNA Automation\Data\{filename}'

Info, freq, field, RefS11, S11 = Load.NPZ_1P(dir)
```

### 1.4 S11 Plots

Visualize S11 plots at a specified frequency.

```python
specified_freq = 3
Plot.S_parameter(freq, field, specified_freq, S11)
```

### 1.5 Set Frequency and Field Bounds

Set frequency and field bounds for further analysis.

```python
lower_freq = 0.5
upper_freq = 4
lower_field = 0
upper_field = 200
freq_used = Crop.cut_freq(freq, lower_freq, upper_freq)[0]
# ...
```

### 1.6 Visualizing Filtered Data

Visualize raw and filtered data.

```python
abspow = Calc.AbsPower1P(S11, RefS11)  # Power Absorbed
filteredData = fil.Filter_Spline(abspow, a=0.1)
# ...
```

### 1.7 Single dP/dH Plot

Generate a single dP/dH plot.

```python
frequency = 7
field = field_used
d = Plot.Fdata(cutData, F=frequency, h=field_used, freq=freq_used)
# ...
```

### 1.8 Shift dP/dH (only considering positive fields)

Shift dP/dH for multiple frequencies.

```python
start = 1
stop = 5
interval = 0.5
frequencies = np.arange(start, stop + interval, interval)
# ...
```

## Section 2: 2 Port Experiment

### 2.1 Initializing 2 Port Experiment

```python
from VNAxKEPCO_2P import Experiment2P

E2 = Experiment2P()
```

### 2.2 Frequency Sweep

```python
ref_field = 500
start_freq = 0.1
stop_freq = 5
step = 20e6  # 20MHz
start_freq, stop_freq = start_freq * 1e9, stop_freq * 1e9
num_points = int((stop_freq - start_freq) / step) + 1

fields = np.linspace(-350, 350, 301)

sample = 'SPT7'
file_name = f'2Port_{sample}_{round(start_freq/1e9, 1)}-{round(stop_freq/1e9, 1)}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\physlab\Desktop\VNA Automation\Data\2port\{file_name}'

# Run the experiment
E2.MeasureRef(ref_field, start_freq, stop_freq, num_points, dir)
E2.Measure(start_freq, stop_freq, num_points, fields, dir)
```

### 2.3 Data Loading

```python
dir = r'c:\Users\plyslab\Desktop\VNA Automation\Data\2port\{filename}'

Info, freq, field, RefS11, RefS21, RefS22, RefS12, S11, S12, S21, S22 = Load.NPZ_2P(dir)
```

### 2.4 S-Parameter Plots (S11 & S21)

```python
specified_freq = 2
Plot.S_parameter(freq, field, S11, S21)
```

### 2.5 Set Frequency and Field Bounds

```python
lower_freq = 0.5
upper_freq = 5
lower_field = -350
upper_field = 350
freq_used = Crop.cut_freq(freq, lower_freq, upper_freq)[0]
# ...
```

### 2.6 Visualize Raw & Filtered Plots

```python
abspow = Calc.AbsPower2P(S11, S21, RefS11, RefS21)
filteredData = fil.Filter_Spline(abspow, a=0.1)
# ...
```

### 2.7 Line Plots For Specific Field and Frequency

Generate line plots for specific field and frequency.

```python
# Field
plt.figure(figsize=(6, 3))
Plot.plotH(cutData, H=100, f=freq_used, field=field_used)
# ...

# Frequency
plt.figure(figsize=(6, 3))
Plot.plotF(cutData, F=2, h=field_used, freq=freq_used)
# ...
```

### 2.8 Single dP/dH Plot

Generate a single dP/dH plot for the 2 Port experiment.

```python
frequency = 2 * 1e9
field_below = 80
field_above = 80
lower = Calc.concentrated_dpdh(cutData, frequency, field_used, freq_used * 1e9, field_below, field_above)[0]
# ...
```

### 2.9 Shift dP/dH (only considering positive fields)

Shift dP/dH for multiple frequencies in the 2 Port experiment.

```python
start = 2
stop = 5
interval = 0.1
frequencies = np.arange(start, stop + interval, interval)
# ...
```

This tutorial covers the key steps for acquiring and analyzing both 1 Port and 2 Port experiment data. Feel free to explore and modify the provided script based on your specific experimental setup and requirements.

Developer: Mahad Naveed
Supervisor: Dr. Sabieh Anwar
Mentor: Dr. Adnan Raza